# Random Forest

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("dataset/CL-dataset.csv",sep='\t',decimal=",",index_col=0)
df = df.drop(columns=["CustomerID"])

In [31]:
df.head()

,Imax,E,PIL,Mb,MeanD,EDate,class
0,12,4.335643,2.567,156.860294,0.000000,0.000000,2
1,32,6.503112,2.567,356.232222,35.000000,2.226424,2
2,360,6.504979,2.752,442.969333,24.266667,2.235084,2
3,80,4.494680,2.567,189.650000,55.400000,1.177965,1
4,32,0.000000,2.567,292.000000,13.000000,0.918296,1


In [32]:
from sklearn.model_selection import train_test_split
label = df.pop('class')
train_set, test_set, train_label, test_label = train_test_split(df, label, stratify =label, test_size=0.30)

In [33]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


## Grid search

In [11]:


param_dist = [{"max_depth": [2,3,12,None],
              "max_features": [6],# Al massimo 6 non di più
              "min_samples_split": [10,50],
              "min_samples_leaf": [10,50],
              "bootstrap": [True, False],
              "criterion": ["entropy", "gini"],
              "class_weight":['balanced', None, {0: 0.3, 1: 0.7}]}]

clf = RandomForestClassifier(n_estimators=30)
grid_search = GridSearchCV(clf, param_dist,  
                            n_jobs=3, 
                            scoring='accuracy')

grid_search.fit(train_set, train_label)
print("Best parameters:",grid_search.best_params_)
print("Best Score: ",grid_search.best_score_)

print("Complete Grid search:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

## Modello Finale

In [34]:
rf = RandomForestClassifier(n_estimators=30, 
                             criterion='gini',
                             max_features=6,
                             max_depth=6, 
                             min_samples_split=32,
                             min_samples_leaf=10,
                             bootstrap=False) 
rf = rf.fit(train_set, train_label)

In [35]:
#prediction on the test test
test_pred_rf = rf.predict(test_set)

In [36]:
#compute the performance of the model
print(classification_report(test_label, 
                            test_pred_rf))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91       859
           1       0.76      0.77      0.76       529
           2       0.80      0.81      0.81       285

    accuracy                           0.85      1673
   macro avg       0.83      0.83      0.83      1673
weighted avg       0.85      0.85      0.85      1673

